# TimeMetamorphy: An Object Evolution Generator

Explore how everyday objects evolved over time. Enter an object name like "phone", "car", or "bicycle" and see its past, present, and future visualized with AI!

To get started, Add the following variable/keys:


*   HF_TOKEN
*   openai_key



### Install the required package

In [ ]:
!pip install transformers transformers[agents] datasets langchain sentence-transformers faiss-cpu  duckduckgo-search langchain-community gradio  pillow smolagents diffusers gradio_client

In [ ]:
!pip install -q huggingface_hub

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import tempfile
import gradio as gr
from smolagents import CodeAgent, InferenceClientModel
from smolagents import DuckDuckGoSearchTool, Tool
from diffusers import DiffusionPipeline
import torch

from huggingface_hub import login
import os


token = os.environ.get("HF_TOKEN")
from google.colab import userdata
token=userdata.get("HF_TOKEN")
token_openai=userdata.get("openai_key")
if token:
    login(token=token)
else:
    print("Warning: HF_TOKEN not set. You may not be able to access private models or tools.")


## Functions and Utils

In [ ]:
# =========================================================
# Utility functions
# =========================================================

def add_label_to_image(image, label):
    draw = ImageDraw.Draw(image)
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    font_size = 30
    try:
        font = ImageFont.truetype(font_path, font_size)
    except:
        font = ImageFont.load_default()

    text_bbox = draw.textbbox((0, 0), label, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    position = (image.width - text_width - 20, image.height - text_height - 20)

    rect_margin = 10
    rect_position = [
        position[0] - rect_margin,
        position[1] - rect_margin,
        position[0] + text_width + rect_margin,
        position[1] + text_height + rect_margin,
    ]
    draw.rectangle(rect_position, fill=(0, 0, 0, 128))
    draw.text(position, label, fill="white", font=font)
    return image


def plot_and_save_agent_image(agent_image, label, save_path=None):
    #pil_image = agent_image.to_raw()
    pil_image = agent_image
    labeled_image = add_label_to_image(pil_image, label)
    #labeled_image.show()
    if save_path:
        labeled_image.save(save_path)
        print(f"Image saved to {save_path}")
    else:
        print("No save path provided. Image not saved.")


def generate_prompts_for_object(object_name):
    return {
        "past": f"Show an old version of a {object_name} from its early days.",
        "present": f"Show a {object_name} with current features/design/technology.",
        "future": f"Show a futuristic version of a {object_name}, by predicting advanced features and futuristic design."
    }


In [ ]:

from PIL import Image

def generate_object_history(object_name):
    images = []
    prompts = generate_prompts_for_object(object_name)

    general_instruction = (
        "Search the necessary information and features for the following prompt, "
        "then generate an image of it."
    )

    image_paths = []
    for time_period, prompt in prompts.items():
        print(f"Generating {time_period} frame: {prompt}")

        try:
            result = agent.run(
                general_instruction,
                additional_args={"prompt": prompt,
                                "width": 256,             # specify width
                                "height": 256,            # specify height
                                "seed": 0,                # optional seed
                                "randomize_seed": False,  # optional
                                "num_inference_steps": 4  # optional
                                }
            )


            # result is tuple: (filepath, seed)
            if isinstance(result, (list, tuple)):
                image_filepath = result[0]
            else:
                image_filepath = result  # fallback in case result is just a string

            # Open the image from filepath
            image = Image.open(image_filepath)

            # Save the image to your naming convention
            image_filename = f"{object_name}_{time_period}.png"
            image.save(image_filename)

            # Optional: call your plotting function (if needed)
            plot_and_save_agent_image(image, f"{object_name} - {time_period.title()}", save_path=image_filename)

            image_paths.append(image_filename)
            images.append(image)

        except Exception as e:
            print(f"Agent failed on {time_period}: {e}")
            continue

    # Create GIF from generated images if any
    gif_path = f"{object_name}_evolution.gif"
    if images:
        images[0].save(gif_path, save_all=True, append_images=images[1:], duration=1000, loop=0)

    return image_paths, gif_path

## Agent Creating and tools

In [ ]:

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
	api_name="/infer",
    name="image_generator",
    description="Generate an image from a prompt"
)


# =========================================================
# Tool and Agent Initialization
# =========================================================

search_tool = DuckDuckGoSearchTool()

#llm_engine = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
#agent = CodeAgent(tools=[image_generation_tool, search_tool], model=llm_engine)


Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


In [ ]:
from smolagents import OpenAIServerModel

# Inicialização do modelo OpenAI com smolagents
llm_engine = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=token_openai,
)

agent = CodeAgent(tools=[image_generation_tool, search_tool], model=llm_engine)

## Test/Try Agent

In [ ]:
image_gallery, gif_output=generate_object_history('car')

Generating past frame: Show an old version of a car from its early days.


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search the necessary information and features for the following prompt, then generate an image of it.           │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'prompt': 'Show an old version of a car from its early days.', 'width': 256, 'height': 256, 'seed': 0,         │
│ 'randomize_seed': False, 'num_inference_steps': 4}.                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  car_info = web_search(query="features of early cars")                                                            
  print(car_info)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[History of the automobile - Wikipedia](https://en.wikipedia.org/wiki/History_of_the_automobile)
The Ford Model T (foreground) and Volkswagen Beetle (background) are among the most mass-produced car models in 
history.. Crude ideas and designs of automobiles can be traced back to ancient and medieval times. [1] [2] In 1649,
Hans Hautsch of Nuremberg built a clockwork-driven carriage.[1] [3] In 1672, a small-scale steam-powered vehicle 
was created by Ferdinand Verbiest; [4] the first steam ...

[features of some early cars Crossword Clue | 
Wordplays.com](https://www.wordplays.com/crossword-solver/features-of-some-early-cars)
Answers for features of some early cars crossword clue, 5 letters. Search for crossword clues found in the Daily 
Celebrity, NY Times, Daily Mirror, Telegraph and major publications. Find clues for features of some early cars or 
most any crossword answer or clues for crossword answers.

[Automobile - Invention, Evolution, Impact | 
Britannica](https://www.britannica.com/technology/automobile/History-of-the-automobile)
Automobile - Invention, Evolution, Impact: Unlike many other major inventions, the original idea of the automobile 
cannot be attributed to a single individual. The idea certainly occurred long before it was first recorded in the 
Iliad, in which Homer (in Alexander Pope's translation) states that Vulcan in a single day made 20 tricycles, which
Leonardo da Vinci considered the idea of a self ...

[Early Cars: Fact Sheet for Kids - Smithsonian Institution](https://www.si.edu/spotlight/early-cars)
Beginning in the 1770s, many people tried to make cars that would run on steam. Some early steam cars worked well, 
and some did not. Some were fire pumpers that moved by themselves, and others were small locomotives with road 
wheels. Beginning in the 1880s, inventors tried very hard to make cars that would run well enough to use every day.

[How Cars Started Back in the Early Days - Car and 
Driver](https://www.caranddriver.com/features/g15376936/how-cars-started/)
There were all kinds of things to do to get a car running smoothly: setting points, adjusting chokes, adding oil 
and additives, even cranking the engine by hand in the early days. By yesterday's ...

[History of the automobile - Wikicars](https://wikicars.org/en/History_of_the_automobile)
The history of the automobile begins as early as 1769, with the creation of steam-powered automobiles capable of 
human transportEckermann, Erik (2001). World History of the Automobile. SAE Press, p.14. In 1806, the first cars 
powered by internal combustion engines running on fuel gas appeared, which led to the introduction in 1885 of the 
ubiquitous modern gasoline- or petrol-fueled ...

[Automobile History: When Were Cars Invented? - 
CarParts.com](https://www.carparts.com/blog/automobile-history-when-were-cars-invented/)
Bosch registered a patent for a mechanism to prevent wheel locking as early as 1936, but the technology was 
hampered by slow mechanical controls. Semiconductor technology in the early 1960s paved the way for faster, 
electronically controlled systems. Bosch unveiled ABS 2 in 1978, the first production-ready electronically 
controlled ABS.

[History of cars. - Verily Facts](https://verilyfacts.com/history-of-cars/)
The history of cars is a fascinating journey that spans over centuries, evolving from basic steam-powered vehicles 
to the advanced and diverse automobiles we see on the roads today. Here's an overview of the key milestones in the 
history of cars, highlighting different types and kinds of vehicles: 1. Early Experiments (Late 18th Century - 19th
Century): 1770s-1780s: Nicolas-Joseph Cugnot, a ...

[How Cars Have Changed and Improved Over Time - I Drive 
Safely](https://www.idrivesafely.com/defensive-driving/trending/evolution-automobile)
In the late '80s and early '90s, keyless entry systems, electric doors and windows, sunroofs, and CD players all 
became standard features. Technology

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  image = image_generator(prompt="A futuristic version of a car featuring electric and hybrid technology,          
  autonomous driving capabilities, sleek aerodynamic design, advanced connectivity, and reconfigurable             
  interiors.", width=256, height=256, seed=0, randomize_seed=False, num_inference_steps=4)                         
  final_answer(image)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: /tmp/gradio/a6983e49390514ed51d3b6056df8f23aebf7e695ba8cae2561c35b3e526f74e6/image.webp

[Step 2: Duration 4.31 seconds| Input tokens: 5,272 | Output tokens: 273]

Image saved to car_future.png


## Gradio App Integration

In [ ]:

# =========================================================
# Gradio Interface
# =========================================================

def create_gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# TimeMetamorphy: An Object Evolution Generator")
        gr.Markdown("""
        Explore how everyday objects evolved over time. Enter an object name like "phone", "car", or "bicycle"
        and see its past, present, and future visualized with AI!
        """)

        default_images = [
            "car_past.png",
            "car_present.png",
            "car_future.png"
        ]
        default_gif_path = "car_evolution.gif"

        with gr.Row():
            with gr.Column():
                object_name_input = gr.Textbox(label="Enter an object name", placeholder="e.g. bicycle, car, phone")
                generate_button = gr.Button("Generate Evolution")
                image_gallery = gr.Gallery(label="Generated Images", columns=3, rows=1, value=default_images, type="filepath")
                gif_output = gr.Image(label="Generated GIF", value=default_gif_path, type="filepath")
                #image_gallery = gr.Gallery(label="Generated Images", columns=3, rows=1,  type="filepath")
                #gif_output = gr.Image(label="Generated GIF", type="filepath")

        generate_button.click(fn=generate_object_history, inputs=[object_name_input], outputs=[image_gallery, gif_output])

    return demo


# Launch the interface
demo = create_gradio_interface()
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ce8889716a234660f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
